In [1]:
import sqlite3
import zipfile

import pandas as pd

## Conhecer a base de dados dos acórdãos do TCU

A base de dados dos acórdãos do TCU foi disponibilizada para download público na plataforma Kaggle. Os dados dizem respeito aos acórdãos proferidos pelo TCU entre os anos de 1992 até 30/08/2019.
Acesse o endereço <https://www.kaggle.com/ferraz/acordaos-tcu> e baixe o arquivo no link Download (4GB) e salve na pasta '/dados'.
github: <https://github.com/netoferraz/acordaos-tcu>

In [ ]:
with zipfile.ZipFile('dados/330881_660826_bundle_archive.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [2]:
# Cria conexão com a base de datos sqlite
conn = sqlite3.connect('dados/tcu-acordaos.db')

In [30]:
# Cria o dataframe df_acordaos
df_acordaos = pd.read_sql_query('SELECT * from acordaos', conn)

In [31]:
print(df_acordaos.shape, '\n')

# Mostra as colunas da base
print(df_acordaos.columns, '\n')

print(df_acordaos.info(), '\n')

(298942, 20) 

Index(['id', 'urn', 'ano_acordao', 'numero_acordao', 'relator', 'processo',
       'tipo_processo', 'data_sessao', 'numero_ata',
       'interessado_reponsavel_recorrente', 'entidade', 'representante_mp',
       'unidade_tecnica', 'repr_legal', 'assunto', 'sumario', 'acordao',
       'quorum', 'relatorio', 'voto'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298942 entries, 0 to 298941
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   id                                 298942 non-null  int64 
 1   urn                                298942 non-null  object
 2   ano_acordao                        298942 non-null  int64 
 3   numero_acordao                     298123 non-null  object
 4   relator                            298470 non-null  object
 5   processo                           298562 non-null  object
 6   tipo_processo   

In [32]:
# Transfora em string as colunas ano_acordao
df_acordaos['ano_acordao'] = df_acordaos['ano_acordao'].map(str)

In [33]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['representante_mp', 'assunto', 'unidade_tecnica', 'repr_legal', 'quorum',
                               'interessado_reponsavel_recorrente'], axis=1)

In [34]:
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298942 entries, 0 to 298941
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              298942 non-null  int64 
 1   urn             298942 non-null  object
 2   ano_acordao     298942 non-null  object
 3   numero_acordao  298123 non-null  object
 4   relator         298470 non-null  object
 5   processo        298562 non-null  object
 6   tipo_processo   298470 non-null  object
 7   data_sessao     298562 non-null  object
 8   numero_ata      298470 non-null  object
 9   entidade        297777 non-null  object
 10  sumario         269290 non-null  object
 11  acordao         298447 non-null  object
 12  relatorio       269199 non-null  object
 13  voto            269199 non-null  object
dtypes: int64(1), object(13)
memory usage: 31.9+ MB


In [35]:
print('Qtde de acordãos sem número de processo: ', df_acordaos[df_acordaos.processo.isnull()].shape[0])
print('Qtde de acordãos com tipo de processo SIGILOSO: ', df_acordaos[(df_acordaos.tipo_processo == 'SIGILOSO')].shape[0])
print('Qtde de acordãos com acordao SIGILOSO: ', df_acordaos[(df_acordaos.acordao == 'SIGILOSO') | (df_acordaos.acordao == 'None')].shape[0])
print('Qtde de acordãos com voto SIGILOSO: ', df_acordaos[(df_acordaos.voto == 'SIGILOSO')  | (df_acordaos.voto == 'None')].shape[0])
print('Qtde de acordãos com entidade SIGILOSO: ', df_acordaos[(df_acordaos.entidade == 'SIGILOSO')  | (df_acordaos.entidade == 'None')].shape[0])
print('Qtde de acordãos com Número de Acórdão não utilizado: ', df_acordaos[df_acordaos.sumario == 'Número de Acórdão não utilizado'].shape[0])

Qtde de acordãos sem número de processo:  380
Qtde de acordãos com tipo de processo SIGILOSO:  5201
Qtde de acordãos com acordao SIGILOSO:  1787
Qtde de acordãos com voto SIGILOSO:  2597
Qtde de acordãos com entidade SIGILOSO:  5542
Qtde de acordãos com Número de Acórdão não utilizado:  92


In [39]:
df_acordaos[(df_acordaos['entidade'] == 'SIGILOSO') | (df_acordaos['tipo_processo'].str.upper == 'SIGILOGO')
       | (df_acordaos['sumario'].str.upper == 'SIGILOGO')
       | (df_acordaos['acordao'].str.upper == 'SIGILOGO')
       | (df_acordaos['relatorio'].str.upper == 'SIGILOGO')
       | (df_acordaos['entidade'].str.upper == 'SIGILOGO')
       | (df_acordaos['voto'].str.upper == 'SIGILOGO')
        | (df_acordaos.sumario == 'Número de Acórdão não utilizado')].shape[0]

5538

In [40]:
df_temp = df_acordaos

In [ ]:
del df_acordaos
df_acordaos = df_temp

In [41]:
df_acordaos = df_acordaos[df_acordaos.processo.notnull()]
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[df_acordaos['tipo_processo'] =='SIGILOSO'])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['acordao'] == 'SIGILOSO') | (df_acordaos['acordao'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['voto'] == 'SIGILOSO') | (df_acordaos['voto'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['entidade'] == 'SIGILOSO') | (df_acordaos['entidade'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['sumario'] == 'Número de Acórdão não utilizado')])
print(df_acordaos.shape[0])

298562
293361
293087
292277
292207
292115


In [42]:
print('Qtde de acordãos sem número de processo: ', df_acordaos[df_acordaos.processo.isnull()].shape[0])
print('Qtde de acordãos com tipo de processo SIGILOSO: ', df_acordaos[(df_acordaos.tipo_processo == 'SIGILOSO')].shape[0])
print('Qtde de acordãos com acordao SIGILOSO: ', df_acordaos[(df_acordaos.acordao == 'SIGILOSO') | (df_acordaos.acordao == 'None')].shape[0])
print('Qtde de acordãos com voto SIGILOSO: ', df_acordaos[(df_acordaos.voto == 'SIGILOSO')  | (df_acordaos.voto == 'None')].shape[0])
print('Qtde de acordãos com entidade SIGILOSO: ', df_acordaos[(df_acordaos.entidade == 'SIGILOSO')  | (df_acordaos.entidade == 'None')].shape[0])
print('Qtde de acordãos com Número de Acórdão não utilizado: ', df_acordaos[df_acordaos.sumario == 'Número de Acórdão não utilizado'].shape[0])

Qtde de acordãos sem número de processo:  0
Qtde de acordãos com tipo de processo SIGILOSO:  0
Qtde de acordãos com acordao SIGILOSO:  0
Qtde de acordãos com voto SIGILOSO:  0
Qtde de acordãos com entidade SIGILOSO:  0
Qtde de acordãos com Número de Acórdão não utilizado:  0


In [44]:
df_acordaos[(df_acordaos['entidade'] == 'SIGILOSO') | (df_acordaos['tipo_processo'].str.upper == 'SIGILOGO')
       | (df_acordaos['sumario'].str.upper == 'SIGILOGO')
       | (df_acordaos['acordao'].str.upper == 'SIGILOGO')
       | (df_acordaos['relatorio'].str.upper == 'SIGILOGO')
       | (df_acordaos['voto'].str.upper == 'SIGILOGO')
        | (df_acordaos.sumario == 'Número de Acórdão não utilizado')].shape[0]

0

In [ ]:
# Cria coluna "decisao" contendo a união dos textos das colunas "relatorio", "voto" e "acordao"
# df_acordaos['decisao'] = df_acordaos.relatorio + ' ' + df_acordaos.voto + ' ' + df_acordaos.acordao

In [ ]:
# Salva o arquivo em um csv compactado
df_acordaos.to_csv('dados/df_acordaos.csv', sep='|', index=False, compression='gzip')

In [ ]:
df_temp = df_acordaos[df_acordaos.duplicated(['numero_acordao', 'processo'])]
# duplicateDFRow = dfObj[dfObj.duplicated(['Name', 'Seasons'])]

In [ ]:
df_temp.fillna(method='ffill', inplace=True)
df_temp.shape

In [ ]:
print('Qtde de acordãos com entidade SIGILOSO: ', df_temp[(df_temp.processo == '000.000/0000-0')].shape[0])

In [ ]:
print('Qtde de acordãos com entidade SIGILOSO: ', df_temp[(df_temp.sumario == 'Número de Acórdão não utilizado')].shape[0])

In [26]:
df_acordaos[(df_acordaos.sumario == 'Número de Acórdão não utilizado') | (df_acordaos.voto.isna != True)]

,id,urn,ano_acordao,numero_acordao,relator,processo,tipo_processo,data_sessao,numero_ata,entidade,sumario,acordao,relatorio,voto
0,1,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2000,1,WALTON ALENCAR RODRIGUES,300.002/1997-1 007.885/1999-7 008.166/1999-4 0...,,25/01/2000,01/2000-Primeira Câmara,,,ACÓRDÃO: Os Ministros do Tribunal de Contas da...,,
1,2,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2000,2,JOSÉ ANTÔNIO BARRETO DE MACEDO,012.895/1994-6,,25/01/2000,01/2000-Primeira Câmara,,,ACÓRDÃO: Os Ministros do Tribunal de Contas da...,,
2,3,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2000,5,HUMBERTO GUIMARÃES SOUTO,006.380/1999-9 007.394/1999-3,,15/02/2000,04/2000-Primeira Câmara,,,ACÓRDÃO: Os Ministros do Tribunal de Contas da...,,
3,4,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2000,14,WALTON ALENCAR RODRIGUES,008.112/1999-1 250.213/1998-2,,14/03/2000,07/2000-Primeira Câmara,,,ACÓRDÃO: Os Ministros do Tribunal de Contas da...,,
4,5,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2000,22,WALTON ALENCAR RODRIGUES,004.338/1998-7,,04/04/2000,10/2000-Primeira Câmara,,,ACÓRDÃO: Os Ministros do Tribunal de Contas da...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298936,298937,NA,2018,2980,AUGUSTO NARDES,029.826/2014-4,Tomada de contas especial instaurada em decorr...,12/12/2018,50/2018-Plenário,Município de Sítio Novo - RN.,TOMADA DE CONTAS ESPECIAL. CONVÊNIO. FUNASA. E...,"VISTOS, relatados e discutidos estes autos de ...",Trata-se de tomada de contas especial instaura...,Trata-se de tomada de contas especial instaura...
298937,298938,NA,2018,3071,AUGUSTO NARDES,015.920/2016-0,,12/12/2018,24/2018-Plenário,,,ACÓRDÃO Nº 3071/2018 - TCU - Plenário Os Minis...,,
298938,298939,NA,2018,3007,AUGUSTO NARDES,040.876/2018-7,,12/12/2018,50/2018-Plenário,Ministério da Saúde,,ACÓRDÃO Nº 3007/2018 - TCU - PlenárioOs Minist...,,
298939,298940,NA,2018,3072,AUGUSTO NARDES,032.098/2015-4,,12/12/2018,24/2018-Plenário,Departamento Nacional de Infraestrutura de Tra...,,ACÓRDÃO Nº 3072/2018 - TCU Plenário Os Minist...,,


In [27]:
df_acordaos[(df_acordaos.sumario == 'Número de Acórdão não utilizado')]

,id,urn,ano_acordao,numero_acordao,relator,processo,tipo_processo,data_sessao,numero_ata,entidade,sumario,acordao,relatorio,voto
182523,182524,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2014,None,None,000.000/0000-0,None,28/01/2014,None,None,Número de Acórdão não utilizado,None,None,None
182524,182525,urn:lex:br:tribunal.contas.uniao;plenario:acor...,2014,None,None,000.000/0000-0,None,29/01/2014,None,None,Número de Acórdão não utilizado,None,None,None
182525,182526,urn:lex:br:tribunal.contas.uniao;plenario:acor...,2014,None,None,000.000/0000-0,None,12/02/2014,None,None,Número de Acórdão não utilizado,None,None,None
182526,182527,urn:lex:br:tribunal.contas.uniao;plenario:acor...,2014,None,None,000.000/0000-0,None,12/02/2014,None,None,Número de Acórdão não utilizado,None,None,None
182527,182528,urn:lex:br:tribunal.contas.uniao;plenario:acor...,2014,None,None,000.000/0000-0,None,12/02/2014,None,None,Número de Acórdão não utilizado,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182610,182611,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2014,None,None,000.000/0000-0,None,10/12/2014,None,None,Número de Acórdão não utilizado,None,None,None
182611,182612,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2014,None,None,000.000/0000-0,None,10/12/2014,None,None,Número de Acórdão não utilizado,None,None,None
182612,182613,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2014,None,None,000.000/0000-0,None,10/12/2014,None,None,Número de Acórdão não utilizado,None,None,None
182613,182614,urn:lex:br:tribunal.contas.uniao;camara.1:acor...,2014,None,None,000.000/0000-0,None,10/12/2014,None,None,Número de Acórdão não utilizado,None,None,None


In [ ]:
df_temp.sort_values(by=['numero_acordao'], ascending=False)

In [ ]:
duplicados = df_acordaos[df_acordaos['processo'].duplicated()]
# duplicateDFRow = dfObj[dfObj.duplicated()]

In [ ]:
duplicados = df_acordaos[df_acordaos.duplicated(['numero_acordao', 'processo', 'data_sessao'])]

In [ ]:
vazios = df_acordaos[(df_acordaos['numero_acordao']=='None') & (df_acordaos['']) & ()]

In [ ]:
df_acordaos[(df_acordaos['numero_acordao']=='None') & (df_acordaos['']) & ()]

In [ ]:
df_acordaos[(df_acordaos['acordao'] == 'SIGILOSO') | (df_acordaos['acordao'] == 'None')].shape

In [ ]:
# Lê o arquivo de um csv compactado
df_acordaos = pd.read_csv('dados/df_acordaos.csv', sep='|', compression='gzip')